In [13]:
import sys
!{sys.executable} -m pip install -r requirements.txt
sys.path.append('C:\\Users\\Ben\\AppData\\Local\\Programs\\Python\\Python38\\Scripts')
import pandas as pd
import numpy as np
import glob
import torch
import torch.nn as nn
from gensim.models import Word2Vec
from gensim.test.utils import common_texts
import re
import string
import math
import random
from tqdm import tqdm
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, roc_curve, auc,plot_roc_curve,accuracy_score
from glove import Corpus, Glove



C:\Users\Ben\AppData\Local\Programs\Python\Python38\python.exe
     -------------------------------------- 244.4/244.4 KB 1.9 MB/s eta 0:00:00
     --------------------------------------- 24.0/24.0 MB 24.3 MB/s eta 0:00:00
     ---------------------------------------- 7.2/7.2 MB 23.0 MB/s eta 0:00:00
     ---------------------------------------- 1.5/1.5 MB 23.5 MB/s eta 0:00:00
     --------------------------------------- 10.6/10.6 MB 28.4 MB/s eta 0:00:00
     ------------------------------------- 158.0/158.0 MB 21.9 MB/s eta 0:00:00
     ---------------------------------------- 88.1/88.1 KB ? eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 1.7/1.7 MB 26.2 MB/s eta 0:00:00
     --------------------------------------- 14.7/14.7 MB 31.2 MB/s eta 0:00:00
     --------------------------------------- 36.9/36.9 MB 22.6 MB/s eta 0:00:00
     ----------------------------------------

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\Ben\\AppData\\Local\\Programs\\Python\\Python38\\Lib\\site-packages\\numpy\\.libs\\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll'
Consider using the `--user` option or check the permissions.



ImportError: DLL load failed while importing corpus_cython: The specified module could not be found.

# Reading the data

In [6]:
vocab_path = 'imdb.vocab'

def make_vocab(path):
    # Reading imdb vocab file
    with open(path,encoding="utf8") as file:
        words_list=file.readlines()
    
    # Splitting /n and "" from the words
    words_list=[s.split('\n')[0].split('""')[0] for s in words_list]
    # Generating a list of lists containing the vocab word
    words_list=[[word] for word in words_list]
    return words_list

# A list to hold the imdb vocab
words_list=make_vocab(vocab_path)

In [185]:
# words_list

In [7]:
def read_files(path):
    """
    Run this script from the root directory
    """
    for file in glob.glob(f"{path}\*.txt"):
        yield file
        
def clean_query(query) -> string:
    """Remove punctuation 

    :param query: string to remove punctuation from
    :return: string with no puncuation
    """
#     return query.translate(query.maketrans('','',string.punctuation)).replace('br','')
    for punc in string.punctuation:
        if punc == "'":
            query=query.replace(punc,'')
        else:
            query=query.replace(punc,' ')
    return query.replace('br',' ')


def load_reviews(path) -> dict:
    """
    Load reviews from a given path
    :param path: path of a give review directory
    :return: dictionary of reviews
    """
    
    reviews = {}
    for file_name in read_files(path):
        split=file_name.split('_')
        if len(split)==2:
            review_ind=int(split[0].split('\\')[-1])
            if review_ind not in reviews:
                with open(file_name, encoding='UTF-8') as file:
                    reviews[review_ind]=([clean_query(file.read())],int(split[1].split('.')[0]))
    return reviews

    

def load_positive_reviews() -> dict:
    """Load positive reviews

    :return: _description_
    """
    path = "train/pos"
    return load_reviews(path)

def load_negative_reviews() -> dict:
    path = "train/neg"
    return load_reviews(path)
    
def create_word_counter(reviews):
    reviews_list =reviews
    word_counter = {}
    for review in reviews_list.values():
        review_text = review[0][0]
        for word in review_text.split():
            if word.lower() in word_counter:
                word_counter[word.lower()] +=1
            else:
                word_counter[word.lower()] =1
                
    # Remove words with less than 5 appearances
    words_to_remove = set()
    clean_word_counter = {}
    for word, count in word_counter.items():
        if count < 5: 
            words_to_remove.add(word)
        else:
            clean_word_counter[word] = count

    for review in reviews_list.keys(): 
        review_text = reviews_list[review][0][0]
        text_without_uncommon_words = []
        for word in review_text.split():
            if word not in words_to_remove:
                text_without_uncommon_words.append(word)
        reviews_list[review]=([" ".join(text_without_uncommon_words)],reviews_list[review][1])

        
    return clean_word_counter, reviews_list

def lower(query):
    return " ".join(list(map(lambda x:x.lower(),query.split())))

def concat_sentences(lst,query):
    lst.append(lower(query[0]).split())
    return


wc, clean_reviews = create_word_counter(load_positive_reviews())

print(clean_reviews[0])
print(wc)                    

(['Bromwell High is a cartoon comedy It ran at the same time as some other programs about school life such as Teachers My 35 years in the teaching profession lead me to believe that Bromwell Highs satire is much closer to reality than is Teachers The to survive financially the insightful students who can see right through their pathetic teachers pomp the of the whole situation all remind me of the schools I knew and their students When I saw the episode in which a student repeatedly tried to burn down the school I immediately recalled at High A classic line INSPECTOR Im here to sack one of your teachers STUDENT Welcome to Bromwell High I expect that many adults of my age think that Bromwell High is far fetched What a pity that it isnt'], 9)
{'bromwell': 8, 'high': 1097, 'is': 57254, 'a': 83954, 'cartoon': 247, 'comedy': 1737, 'it': 39397, 'ran': 121, 'at': 11243, 'the': 173315, 'same': 1972, 'time': 6509, 'as': 26313, 'some': 7442, 'other': 5369, 'programs': 41, 'about': 8313, 'school'

In [8]:
sentences=[]
pos_reviews=create_word_counter(load_positive_reviews())[1]
neg_reviews=create_word_counter(load_negative_reviews())[1]

for query in pos_reviews.keys():
    concat_sentences(sentences,pos_reviews[query][0])

for query in neg_reviews.keys():
    concat_sentences(sentences,neg_reviews[query][0])

# sentences[0:2]

# Training and creating Word2Vec Embeddings

In [124]:
def tokenize(query,model):
    return torch.LongTensor([model.wv.get_index(token) for token in query.split()])

def w2vembedding(query:str , model):
    """
    Create embeddings with word2vec

    :param query: string
    :param model: model object
    :return: nn.Embedding
    """
    query=lower(clean_query(query))
    weights = torch.FloatTensor(model.wv.vectors)
    embedding = nn.Embedding.from_pretrained(weights)

    indx=tokenize(query,model)
    return embedding(indx)


# Training a new w2v embedding based on imdb vocab
modelw2v=Word2Vec(sentences=sentences
                  ,vector_size=150
                  ,min_count=1, window=3)

v = w2vembedding("I basically skimmed through the movie but just enough to catch watch the plot was about. To tell you the truth it was kind of boring to me and at some spots it didn't make sense. The only reason I watched this movie in the first place was to see CHACE CRAWFORD!!! He is so hot, but in this movie his hair was kind of weird. But still hot.<br /><br />However, despite how hot CHACE is, it really did not make up for the film. I guess the plot isn't that bad but what really threw me over was the fact that they cuss in like every sentence. Is it that hard to express your anger without saying the F word every time?The cussing was annoying and the whole flashy, camera shaking thing gave me a headache.<br /><br />All in all, although the plot was OK, I found the film to be a bore and over dramatic. That's why I only cut to scenes with CHACE in it. LOL Anyways, not worth renting unless your a die-hard fan of a specific cast member like I was. Oh yeah the cast was Hot. The girls were HOT!!! But CHACE IS THE BEST!!",modelw2v)

In [126]:
# Word Similarities - (sanity check)
modelw2v.wv.most_similar("batman")

[('superman', 0.6993433833122253),
 ('trilogy', 0.6852944493293762),
 ('godzilla', 0.6629911065101624),
 ('dracula', 0.612054705619812),
 ('galactica', 0.5955700278282166),
 ('godfather', 0.5930099487304688),
 ('1970s', 0.5882107615470886),
 ('lotr', 0.5864568948745728),
 ('1978', 0.5836764574050903),
 ('seventies', 0.5835346579551697)]

# Training and creating Glove Embeddings

In [52]:
def glove_train(vocab) -> Glove:
    """
    Train the Glove object

    :param vocab: vocabulary
    :return: trained Glove object
    """
    
    # instantiate the corpus
    glcorpus = Corpus()
    
    # this will create the word co occurence matrix 
    # window - the distance between two words algo should consider to find some relationship between them
    glcorpus.fit(vocab, window=20)

    # instantiate the model
    # no_components - output vector size
    glovemod = Glove(no_components=150, learning_rate=0.05)


    # and fit over the corpus matrix
    glovemod.fit(glcorpus.matrix, epochs=20, no_threads=4)

    # finally we add the vocabulary to the model
    glovemod.add_dictionary(glcorpus.dictionary)
    
    return glovemod

glove_model = glove_train(sentences)

In [54]:
# Word Similarities - (sanity check)
glove_model.most_similar("batman")

[('superman', 0.8333289460099211),
 ('cartoon', 0.7292750156703827),
 ('series', 0.7252292534918698),
 ('animated', 0.7221026513182527)]

In [ ]:
## Functions for glove embeddings

def tokenize_glove(query,model):
    return torch.tensor([glove_model.dictionary[token] for token in query.split()])

def glove_embedding(query,model):
    query=lower(clean_query(query))
    weights = torch.FloatTensor(model.word_vectors)
    embedding = nn.Embedding.from_pretrained(weights)

    indx=tokenize_glove(query,model)
    return embedding(indx)

# Arranging the data

In [125]:
def shuffledata(x_lst, y_lst):
    """
    Shuffling concatenated data to keep the order of the tagging

    :param x_lst: list of samples
    :param y_lst: list of labels
    :return: _description_
    """
    z=list(zip(x_lst,y_lst))
    random.shuffle(z)
    x,y=zip(*z)
    return x,y

# The class helps us to merge the input and labels into a single class which the dataloader operates on
class DLcorrection:
    def __init__(self, reviews, targets,embedding,model):
#         self.reviews=[w2vembedding(review[0],modelw2v) for review in reviews]
        self.reviews=[embedding(review[0],model) for review in reviews]
        self.targets=targets
        
    def __getitem__(self, idx):
        # Get the input features and labels from the current image index
        review = self.reviews[idx]
        targets = self.targets[idx]
        return review,targets     

    def __len__(self):
        return len(self.targets)

# Loading positive and negative reviews
x_pos=[pos_reviews[i][0] for i in range(len(pos_reviews.keys()))]
y_pos=[1 for i in range(len(pos_reviews.keys()))]
x_neg=[neg_reviews[i][0] for i in range(len(neg_reviews.keys()))]
y_neg=[0 for i in range(len(neg_reviews.keys()))]


## Stratifying the train and test sets
x_pos_train, x_pos_test, y_pos_train, y_pos_test = train_test_split(x_pos, y_pos, test_size=0.95, random_state=42)
x_neg_train, x_neg_test, y_neg_train, y_neg_test = train_test_split(x_neg, y_neg, test_size=0.95, random_state=42)

x_combined_train=x_pos_train+x_neg_train
x_combined_test=x_pos_test+x_neg_test
y_combined_train=y_pos_train+y_neg_train
y_combined_test=y_pos_test+y_neg_test

x_final_train,y_final_train=shuffledata(x_combined_train,y_combined_train)
x_final_test,y_final_test=shuffledata(x_combined_test,y_combined_test)

# Pushing the train and test input and labels to a class which helps us to get the right input,label for each image- W2V
train_set_united=DLcorrection(list(x_final_train),list(y_final_train),w2vembedding,modelw2v)
test_set_united=DLcorrection(list(x_final_test),list(y_final_test),w2vembedding,modelw2v)

# Doing the same for glove
train_set_united_glove=DLcorrection(list(x_final_train),list(y_final_train),glove_embedding,glove_model)
test_set_united_glove=DLcorrection(list(x_final_test),list(y_final_test),glove_embedding,glove_model)

# LSTM net class

In [127]:
class lstmnet(nn.Module):
    def __init__(self, input_sz, hidden_sz):
        super().__init__()
        self.input_size = input_sz
        self.hidden_size = hidden_sz
        
        #i_t
        self.W_i = nn.Parameter(torch.Tensor(input_sz, hidden_sz))
        self.U_i = nn.Parameter(torch.Tensor(hidden_sz, hidden_sz))
        
        #f_t
        self.W_f = nn.Parameter(torch.Tensor(input_sz, hidden_sz))
        self.U_f = nn.Parameter(torch.Tensor(hidden_sz, hidden_sz))
        
        #c_t
        self.W_c = nn.Parameter(torch.Tensor(input_sz, hidden_sz))
        self.U_c = nn.Parameter(torch.Tensor(hidden_sz, hidden_sz))
        
        #o_t
        self.W_o = nn.Parameter(torch.Tensor(input_sz, hidden_sz))
        self.U_o = nn.Parameter(torch.Tensor(hidden_sz, hidden_sz))
    
        self.sigmoid=nn.Sigmoid()
        self.fc1=nn.Linear(hidden_sz,1)
        
        self.init_weights()
        
        
    def init_weights(self):
        stdv = 1.0 / math.sqrt(self.hidden_size)
        for weight in self.parameters():
            #weight.data.uniform_(-stdv, stdv)
            nn.init.uniform_(weight, -stdv, stdv)
    
    def forward(self, x, init_states=None):
        
        """
        assumes x.shape represents (batch_size, sequence_size, input_size)
        """
        batchsize, seq_sz, _ = x.size()
        hidden_seq = []
        if init_states is None:
            h_t, c_t = (
                torch.zeros(batchsize, self.hidden_size),
                torch.zeros(batchsize, self.hidden_size),
            )
        else:
            h_t, c_t = init_states
            
        for t in range(seq_sz):
            x_t = x[:, t, :]
            
            # @ Stands for matrix multiplication(inner product)
            
            i_t = torch.sigmoid(x_t @ self.W_i + h_t @ self.U_i)
            f_t = torch.sigmoid(x_t @ self.W_f + h_t @ self.U_f)
            g_t = torch.tanh(x_t @ self.W_c + h_t @ self.U_c)
            o_t = torch.sigmoid(x_t @ self.W_o + h_t @ self.U_o)
            c_t = f_t * c_t + i_t * g_t
            h_t = o_t * torch.tanh(c_t)
            
            
            hidden_seq.append(h_t.unsqueeze(0))
            
        #reshape hidden_seq p/ retornar
        hidden_seq = torch.cat(hidden_seq, dim=0)
        hidden_seq = hidden_seq.transpose(0, 1).contiguous()
        
                
#         self.flatsize=x.shape[1]*x.shape[2]
#         self.fc1=nn.Linear(self.flatsize,1)
        
#         self.vec=hidden_seq.flatten().detach()
        self.vec=h_t
#         print(self.vec)
        
#         out=self.fc1(self.vec)
        out=self.fc1(h_t)[0]
#         out=self.sigmoid(outlin)
        return self.sigmoid(out)
#         return hidden_seq, (h_t, c_t)

# LSTM W2V Train

In [105]:
model=lstmnet(150,150)

# Specify loss function - Binary Cross Entropy
criterion = nn.BCELoss()
# Specify optimizer - Adam
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)


# Loading the train and test set to a generator to save space
# Batch size will be the entire data set of the train and the test data (350,150)
batch_size=1
batch_size_test=1
update_each =0

# num_workers = 
train_loader = torch.utils.data.DataLoader(train_set_united, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test_set_united, batch_size=batch_size_test)

# Number of epochs to train the model
n_epochs = 5
losses_list = []
auc_train_list = []
fpr_train_list = []
tpr_train_list = []


fpr_test_list = []
tpr_test_list = []
auc_test_list = []

for epoch in tqdm(range(1, n_epochs+1)):
    # Monitor training loss
    train_loss = 0.0
    epoch_predicted_labels = []

    ###################
    # Train the model #
    ###################
    model.train()
    for review,rating in train_loader:
        # Clear the gradients of all optimized variables
        optimizer.zero_grad()
        # Create embedding
#         embedding=torch.unsqueeze(review,0)
        embedding=review
        # Forward pass: compute predicted outputs by passing inputs to the model, assuring the values are of type float
        outputs = model(embedding)
#         print(outputs)
        # Calculate the loss
        loss = criterion(outputs, rating.float())
        # Backward pass: compute gradient of the loss with respect to model parameters
#         loss.register_hook(lambda grad: print(grad))
        loss.backward()
        update_each+=1
        # Perform a single optimization step (parameter update)
#         if update_each == 100:
        optimizer.step()
#             update_each = 0
        # Update running training loss
        train_loss += loss.item()
    
    

    
        # Converting the outputs and the labels back from cuda to cpu and from tensor to numpy, so we can use them with the scikit library
#         epoch_labels = rating.cpu().detach().numpy()
#         epoch_outputs = outputs.cpu().detach().numpy()
    
#     # Update losses list
#     losses_list.append(train_loss)
#     fpr_train, tpr_train, _ = roc_curve(list(epoch_labels), list(epoch_outputs))
#     fpr_train_list.append(fpr_train)
#     tpr_train_list.append(tpr_train)
#     auc_score = auc(fpr_train, tpr_train)
#     auc_train_list.append(auc_score)

#     # Canceling the gradient optimizer while running on the validation data set (to save memory)
#     with torch.no_grad():
#       for points,labels in test_loader:
#         # Flatten x,y coordinates and labels, turn it into a tensor, and upload it to the cuda core
#         points = points.view(points.size(0), -1).to(device)
#         labels=labels.view(labels.size(0), -1).to(device)
#         outputs = model(points.float()).to(device)
#         # Converting the outputs and the labels back from cuda to cpu and from tensor to numpy, so we can use them with the scikit library
#         epoch_labels = labels.cpu().detach().numpy()
#         epoch_outputs = outputs.cpu().detach().numpy()

#     fpr_test, tpr_test, _ = roc_curve(list(epoch_labels), list(epoch_outputs))
#     auc_score = auc(fpr_test, tpr_test)
#     auc_test_list.append(auc_score)

    train_loss = train_loss/len(train_loader)
    print('Epoch: {} \tTraining Loss: {:.6f}'.format(
        epoch, 
        train_loss
        ))

 20%|████████████████▌                                                                  | 1/5 [07:04<28:17, 424.34s/it]

Epoch: 1 	Training Loss: 0.733499


 40%|█████████████████████████████████▏                                                 | 2/5 [13:50<20:41, 413.80s/it]

Epoch: 2 	Training Loss: 0.702862


 60%|█████████████████████████████████████████████████▊                                 | 3/5 [21:26<14:25, 432.91s/it]

Epoch: 3 	Training Loss: 0.690380


 80%|██████████████████████████████████████████████████████████████████▍                | 4/5 [30:31<07:57, 477.16s/it]

Epoch: 4 	Training Loss: 0.611589


100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [40:50<00:00, 490.08s/it]

Epoch: 5 	Training Loss: 0.551154


# LSTM Glove Train

In [130]:
model_lstmgl=lstmnet(150,150)

# Specify loss function - Binary Cross Entropy
criterion = nn.BCELoss()
# Specify optimizer - Adam
optimizer = torch.optim.Adam(model_lstmgl.parameters(), lr=0.01)


# Loading the train and test set to a generator to save space
# Batch size will be the entire data set of the train and the test data (350,150)
batch_size=1
batch_size_test=1
update_each =0

# num_workers = 
train_loader = torch.utils.data.DataLoader(train_set_united_glove, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test_set_united_glove, batch_size=batch_size_test)

# Number of epochs to train the model
n_epochs = 5
losses_list = []
auc_train_list = []
fpr_train_list = []
tpr_train_list = []


fpr_test_list = []
tpr_test_list = []
auc_test_list = []

for epoch in tqdm(range(1, n_epochs+1)):
    # Monitor training loss
    train_loss = 0.0
    epoch_predicted_labels = []

    ###################
    # Train the model #
    ###################
    model_lstmgl.train()
    for review,rating in train_loader:
        # Clear the gradients of all optimized variables
        optimizer.zero_grad()
        # Create embedding
#         embedding=torch.unsqueeze(review,0)
        embedding=review
        # Forward pass: compute predicted outputs by passing inputs to the model, assuring the values are of type float
        outputs = model_lstmgl(embedding)
#         print(outputs)
        # Calculate the loss
        loss = criterion(outputs, rating.float())
        # Backward pass: compute gradient of the loss with respect to model parameters
#         loss.register_hook(lambda grad: print(grad))
        loss.backward()
        update_each+=1
        # Perform a single optimization step (parameter update)
#         if update_each == 100:
        optimizer.step()
#             update_each = 0
        # Update running training loss
        train_loss += loss.item()
    
    

    
        # Converting the outputs and the labels back from cuda to cpu and from tensor to numpy, so we can use them with the scikit library
#         epoch_labels = rating.cpu().detach().numpy()
#         epoch_outputs = outputs.cpu().detach().numpy()
    
#     # Update losses list
#     losses_list.append(train_loss)
#     fpr_train, tpr_train, _ = roc_curve(list(epoch_labels), list(epoch_outputs))
#     fpr_train_list.append(fpr_train)
#     tpr_train_list.append(tpr_train)
#     auc_score = auc(fpr_train, tpr_train)
#     auc_train_list.append(auc_score)

#     # Canceling the gradient optimizer while running on the validation data set (to save memory)
#     with torch.no_grad():
#       for points,labels in test_loader:
#         # Flatten x,y coordinates and labels, turn it into a tensor, and upload it to the cuda core
#         points = points.view(points.size(0), -1).to(device)
#         labels=labels.view(labels.size(0), -1).to(device)
#         outputs = model(points.float()).to(device)
#         # Converting the outputs and the labels back from cuda to cpu and from tensor to numpy, so we can use them with the scikit library
#         epoch_labels = labels.cpu().detach().numpy()
#         epoch_outputs = outputs.cpu().detach().numpy()

#     fpr_test, tpr_test, _ = roc_curve(list(epoch_labels), list(epoch_outputs))
#     auc_score = auc(fpr_test, tpr_test)
#     auc_test_list.append(auc_score)

    train_loss = train_loss/len(train_loader)
    print('Epoch: {} \tTraining Loss: {:.6f}'.format(
        epoch, 
        train_loss
        ))

  0%|                                                                                            | 0/5 [04:25<?, ?it/s]


KeyboardInterrupt: 

# Net Alternatives - Erase only at the end

In [ ]:
# class lstmnet(nn.Module):
#     def __init__(self, input_sz, hidden_sz):
#         super().__init__()
#         self.input_size = input_sz
#         self.hidden_size = hidden_sz
        
#         #i_t
#         self.W_i = nn.Linear(self.input_size, self.hidden_size)
#         self.U_i = nn.Linear(input_sz, hidden_sz)
        
#         #f_t
#         self.W_f = nn.Linear(input_sz, hidden_sz)
#         self.U_f = nn.Linear(input_sz, hidden_sz)
        
#         #c_t
#         self.W_c = nn.Linear(input_sz, hidden_sz)
#         self.U_c = nn.Linear(input_sz, hidden_sz)
        
#         #o_t
#         self.W_o = nn.Linear(input_sz, hidden_sz)
#         self.U_o = nn.Linear(input_sz, hidden_sz)
        
#         self.sigmoid=nn.Sigmoid()
#         self.fc1=nn.Linear(hidden_sz,1)
        
#         self.init_weights()
        
        
#     def init_weights(self):
#         stdv = 1.0 / math.sqrt(self.hidden_size)
#         for weight in self.parameters():
#             #weight.data.uniform_(-stdv, stdv)
#             nn.init.uniform_(weight, -stdv, stdv)
    
#     def forward(self,
#                 x,
#                 init_states=None):
        
#         """
#         assumes x.shape represents (batch_size, sequence_size, input_size)
#         """
#         batchsize, seq_sz, _ = x.size()
#         hidden_seq = []
#         if init_states is None:
#             h_t, c_t = (
#                 torch.zeros(batchsize, self.hidden_size,requires_grad=True),
#                 torch.zeros(batchsize, self.hidden_size,requires_grad=True),
#             )
#         else:
#             h_t, c_t = init_states
            
#         for t in range(seq_sz):
#             x_t = x[:, t, :]
            
#             # @ Stands for matrix multiplication(inner product)
            
#             i_t = torch.sigmoid(self.W_i(x_t) + self.U_i(h_t))
#             f_t = torch.sigmoid(self.W_f(x_t) + self.U_f(h_t))
#             g_t = torch.tanh(self.W_c(x_t) + self.U_c(h_t))
#             o_t = torch.sigmoid(self.W_o(x_t) + self.U_o(h_t))
#             c_t = f_t * c_t + i_t * g_t
#             h_t = o_t * torch.tanh(c_t)
            
#             hidden_seq.append(h_t.unsqueeze(0))
#         #reshape hidden_seq p/ retornar
#         hidden_seq = torch.cat(hidden_seq, dim=0)
#         hidden_seq = hidden_seq.transpose(0, 1).contiguous()
        
                
# #         self.flatsize=x.shape[1]*x.shape[2]
# #         self.fc1=nn.Linear(self.flatsize,1)
        
# #         self.vec=hidden_seq.flatten().detach()
#         self.vec=h_t
# #         print(self.vec)
        
# #         out=self.fc1(self.vec)
#         out=self.fc1(h_t)[0]
# #         out=self.sigmoid(outlin)
#         return self.sigmoid(out)
# #         return hidden_seq, (h_t, c_t)

In [ ]:
# class lstmclassification(nn.Module):
#     def __init__(self, input_sz, hidden_sz):
#         super().__init__()
# #         self.h0=torch.randn(1,100)
# #         self.c0=torch.randn(1,100)
# #         self.lstm=nn.LSTM(input_sz,hidden_sz,num_layers)
#         self.lstm=lstmnet(input_sz,hidden_sz)
#         self.sigmoid=nn.Sigmoid()
    
#     def forward(self,query):
#         ## Making sure the number of neurons in the input layer equals to
#         ## (number of words * number of elements in each word vector)
        
#         self.flatsize=query.shape[1]*query.shape[2]
#         self.fc1=nn.Linear(self.flatsize,1)
        
#         self.vec=self.lstm(query)[0].flatten()
#         out=self.fc1(self.vec)
# #         out=self.sigmoid(outlin)

#         return torch.tensor([float(self.sigmoid(out[0]))])